In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
import time

In [24]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [25]:

class CardsDataSet(Dataset):
    
    def __init__(self, csv_file, transform=None, is_train=True):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.is_train = is_train
        print(self.data.columns)


    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.data.iloc[idx]['filepaths']
        image = Image.open(img_path).convert('RGB')
        image = image.resize((224, 224))
        image = np.array(image)
        image = image.astype(np.uint8)
        
        
        
        # Load label if in training mode
        if self.is_train:
            label = torch.tensor(int(self.data.iloc[idx]['class index']), dtype=torch.long)
            sample = {'image': image, 'label': label}
        else:
            sample = {'image': image}


        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample


In [26]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])


In [27]:
train_dataset = CardsDataSet('cards.csv', transform=transform,is_train=True)
print(f"Train Size: {len(train_dataset)}")

Index(['class index', 'filepaths', 'labels', 'card type', 'data set'], dtype='object')
Train Size: 8158


In [28]:
train_loader = DataLoader(train_dataset,batch_size=15, shuffle=True)
print(f"{train_dataset.data['class index'].unique()}")
print("I AM HERE")

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52]
I AM HERE


In [29]:
class CardRecognitionCNN(nn.Module):
    def __init__(self):
        super(CardRecognitionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 53)
        self.fc4 = nn.Linear(53, 53)
        self.fc5 = nn.Linear(53, 53)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)
        # Flatten the output
        x = x.view(-1, 16 * 53 * 53)

        # Fully connected layers
        x = self.fc1(x)
        x = torch.relu(x)

        x = self.fc2(x)
        x = torch.relu(x)

        x = self.fc3(x)
        x = torch.relu(x)

        x = self.fc4(x)
        x = torch.relu(x)
        x = self.dropout(x)


        x = self.fc5(x)
        return x


In [30]:
class OptimizedCardRecognitionCNN(nn.Module):
    def __init__(self, num_classes=53):
        super(OptimizedCardRecognitionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Global Average Pooling
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = torch.mean(x, dim=[2, 3])  # Global Average Pooling
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [31]:
model = OptimizedCardRecognitionCNN().to(device)
criterion = nn.CrossEntropyLoss() # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class. 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:

num_epochs = 60
print(f"Training Started on {device}")
for epoch in range(num_epochs):
    running_loss = 0.0 
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data['image'], data['label']
        inputs, labels = inputs.to(device), labels.to(device)

        
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Epoch: {epoch + 1}, Batch: {i + 1}, "
                  f"Loss: {running_loss / 100:.4f}, "
                  f"Progress: {100 * (epoch + 1) / num_epochs:.2f}% ")
            

            running_loss = 0.0

print("Finished Training")

Training Started on cuda
Epoch: 1, Batch: 100, Loss: 3.9433, Progress: 1.67% 
Epoch: 1, Batch: 200, Loss: 3.8114, Progress: 1.67% 
Epoch: 1, Batch: 300, Loss: 3.6566, Progress: 1.67% 
Epoch: 1, Batch: 400, Loss: 3.5829, Progress: 1.67% 
Epoch: 1, Batch: 500, Loss: 3.5411, Progress: 1.67% 
Epoch: 2, Batch: 100, Loss: 3.4646, Progress: 3.33% 
Epoch: 2, Batch: 200, Loss: 3.4416, Progress: 3.33% 
Epoch: 2, Batch: 300, Loss: 3.4405, Progress: 3.33% 
Epoch: 2, Batch: 400, Loss: 3.4198, Progress: 3.33% 
Epoch: 2, Batch: 500, Loss: 3.3872, Progress: 3.33% 
Epoch: 3, Batch: 100, Loss: 3.3590, Progress: 5.00% 
Epoch: 3, Batch: 200, Loss: 3.3369, Progress: 5.00% 
Epoch: 3, Batch: 300, Loss: 3.3284, Progress: 5.00% 
Epoch: 3, Batch: 400, Loss: 3.3307, Progress: 5.00% 
Epoch: 3, Batch: 500, Loss: 3.3555, Progress: 5.00% 
Epoch: 4, Batch: 100, Loss: 3.3257, Progress: 6.67% 
Epoch: 4, Batch: 200, Loss: 3.3017, Progress: 6.67% 
Epoch: 4, Batch: 300, Loss: 3.2753, Progress: 6.67% 
Epoch: 4, Batch: 400,

KeyboardInterrupt: 

In [ ]:
#thing to test

# early_stopping_patience = 5  # Stop if no improvement for 5 epochs
# best_loss = float('inf')
# best_model = None
# best_epoch = 0
# max_epochs = 64
# stopping_change = 5

# for epoch in range(max_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data['image'], data['label']
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs.float())
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 100 == 99:
#             print(f"Epoch: {epoch + 1}, Batch: {i + 1}, "
#                   f"Loss: {running_loss / 100:.4f}, "
#                   f"Progress: {100 * (epoch + 1) / max_epochs:.2f}% ")
#             running_loss = 0.0

#     # Validate the model
#     model.eval()
#     valid_loss = 0.0
#     with torch.no_grad():
#         for i, data in enumerate(train_loader, 0):
#             inputs, labels = data['image'], data['label']
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs.float())
#             loss = criterion(outputs, labels)
#             valid_loss += loss.item()

#     valid_loss /= len(train_loader)
#     print(f"Validation Loss: {valid_loss:.4f}")

#     if valid_loss < best_loss - stopping_change:
#         best_loss = valid_loss
#         best_model = model.state_dict()
#         best_epoch = epoch
#     elif epoch - best_epoch > early_stopping_patience:
#         print("Early stopping")
#         break

#     model.train()


In [ ]:
# Save the model
torch.save(model.state_dict(), 'model.pth')
print("Model Saved")

Model Saved
